In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Load the dataset
data = pd.read_excel('gpt2_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Scale the features
X_scaled = StandardScaler().fit_transform(X)

# Apply PCA to retain 95% of variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_gb = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_gb, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
70 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
69 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 3, 'learning_rate': 0.1}
Evaluation Metrics: {'Accuracy Mean': 0.5902, 'Accuracy STD': 0.0521, 'Precision Mean': 0.6, 'Precision STD': 0.0561, 'Recall Mean': 0.5874, 'Recall STD': 0.0543, 'F1 Mean': 0.5906, 'F1 STD': 0.0549}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Load the dataset from an Excel file
data = pd.read_excel('gpt2_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]    # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Feature Scaling (important for many models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE (if needed)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Define the AdaBoost model with a shallow decision tree as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=42)

ada = AdaBoostClassifier(
    estimator=base_estimator,
    random_state=42
)

# Step 4: Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds (trees)
    'learning_rate': [0.01, 0.1, 1.0],  # How much each estimator contributes
    'estimator__max_depth': [3, 5, 7]  # Max depth of the decision tree (base estimator)
}

# Step 5: Set up RandomizedSearchCV with StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=param_distributions,
    n_iter=10,
    cv=skf,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Step 6: Fit the model with the best parameters
random_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters found
print("AdaBoost Best Parameters:", random_search.best_params_)

# Step 7: Perform cross-validation on the best AdaBoost model
best_ada = random_search.best_estimator_
cv_score = cross_validate(
    best_ada, X_train_resampled, y_train_resampled, cv=skf, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False
)
# Step 8: Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("AdaBoost Evaluation Metrics:", results)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Best Parameters: {'n_estimators': 200, 'learning_rate': 1.0, 'estimator__max_depth': 5}


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

AdaBoost Evaluation Metrics: {'Accuracy Mean': 0.6912, 'Accuracy STD': 0.0647, 'Precision Mean': 0.701, 'Precision STD': 0.0567, 'Recall Mean': 0.6912, 'Recall STD': 0.0647, 'F1 Mean': 0.6929, 'F1 STD': 0.0617}
